In [23]:
import requests
import pprint
import pandas as pd
import numpy as np
import json
from rdkit import Chem
from tqdm import tqdm

In [2]:
# def get_disease_info(disease_id):
#     url = 'http://herb.ac.cn/chedi/api/?'
#     headers = {
#         'User-Agent': 'User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36',
#     }
#     data = {
#         'func_name': "detail_api",
#         'key_id': disease_id,
#         'label': "Disease",
#         'v': disease_id,
#     }
#     response = requests.post(url = url,headers=headers,json=data,allow_redirects=True)
#     return response.json()

In [3]:
# good_disease_id = list(pd.read_csv('信息表/常见疾病信息（有参考文献）0820.csv',index_col=0)['Disease_id'])

In [4]:
# good_disease_json = []
# for disease_id in tqdm(good_disease_id):
#     good_disease_json.append(get_disease_info(disease_id))
# np.save("常见疾病详细信息（有参考文献）.npy",good_disease_json)

100%|██████████| 494/494 [01:50<00:00,  4.47it/s]


In [2]:
good_disease_json = np.load("常见疾病详细信息（有参考文献）.npy",allow_pickle=True).tolist()

In [3]:
good_disease_json[3]['paper_ingredient_disease'][1:][0]

[{'link': '/Reference/detail/?v=HBREF000184', 'title': 'HBREF000184'},
 {'link': '/Detail/?v=HBIN016408&label=Ingredient', 'title': 'HBIN016408'},
 'apigenin',
 {'link': 'https://pubmed.ncbi.nlm.nih.gov/21550230/', 'title': '21550230'}]

In [5]:
good_disease_json[0].keys()

dict_keys(['target_disease', 'herb_disease', 'summary', 'ingredient_disease', 'paper_herb_disease', 'paper_disease', 'paper_ingredient_disease'])

In [16]:
good_disease_json[0]['paper_ingredient_disease']

[['Paper id', 'Ingredient id', 'Ingredient name', 'PubMed id'],
 [{'link': '/Reference/detail/?v=HBREF000630', 'title': 'HBREF000630'},
  {'link': '/Detail/?v=HBIN040403&label=Ingredient', 'title': 'HBIN040403'},
  'polydatin',
  {'link': 'https://pubmed.ncbi.nlm.nih.gov/28164346/', 'title': '28164346'}]]

In [14]:
disease_id = []
disgenet_id = []
disease_name = []
disease_type = []
herb_id = []
herb_name = []
for disease in good_disease_json:
    paper_herb_nums = len(disease['paper_herb_disease'])-1
    if paper_herb_nums > 0:
        for paper_herb in disease['paper_herb_disease'][1:]:
            disease_id.append(disease['summary'][1][0])
            disgenet_id.append(disease['summary'][1][1]['title'])
            disease_name.append(disease['summary'][1][2])
            disease_type.append(disease['summary'][1][3])
            herb_id.append(paper_herb[1]['title'])
            herb_name.append(paper_herb[2])
            

disease_herb = pd.DataFrame([disease_id, disgenet_id, disease_name, disease_type, herb_id, herb_name]).T
disease_herb.columns = ['Disease_id','DisGENet_id','Disease_name','Disease_type','Herb_id','Herb_name']

In [17]:
disease_id = []
disgenet_id = []
disease_name = []
disease_type = []
compound_id = []
for disease in good_disease_json:
    paper_compound_nums = len(disease['paper_ingredient_disease'])-1
    if paper_compound_nums > 0:
        for paper_ingredient in disease['paper_ingredient_disease'][1:]:
            disease_id.append(disease['summary'][1][0])
            disgenet_id.append(disease['summary'][1][1]['title'])
            disease_name.append(disease['summary'][1][2])
            disease_type.append(disease['summary'][1][3])
            compound_id.append(paper_ingredient[1]['title'])

disease_compound = pd.DataFrame([disease_id, disgenet_id, disease_name, disease_type, compound_id]).T
disease_compound.columns = ['Disease_id','DisGENet_id','Disease_name','Disease_type','Compound_id']

In [25]:
compound_can_smi = list(set(pd.read_csv(r"D:\Herb数据库\version2\edge\Herb_Compound\Herb_Compound_filter 0208.csv",index_col=0)['Canonical_smiles']))
herb_all = list(set(pd.read_csv(r"D:\Herb数据库\version2\edge\Herb_Compound\Herb_Compound_filter 0208.csv",index_col=0)['Herb']))
disease_compound_with_info = pd.merge(disease_compound,pd.read_csv(r"D:\Herb数据库\version2\node\HERB_ingredient_info.txt",sep='\t')[['Ingredient_Smile','Ingredient_id']],
         left_on='Compound_id',right_on='Ingredient_id',how='left').iloc[:,:-1]
can_smi = []
for smi in disease_compound_with_info['Ingredient_Smile']:
    try:
        can_smi.append(Chem.MolToSmiles(Chem.MolFromSmiles(smi),kekuleSmiles=True,isomericSmiles=False))
    except:
        can_smi.append(None)
disease_compound_with_info['Canonical_smiles'] = can_smi

In [28]:
disease_compound_new = disease_compound_with_info.query("Canonical_smiles in @compound_can_smi")
disease_compound_new.drop('Ingredient_Smile',axis=1,inplace=True)

C:\Users\Dell\AppData\Local\Temp\ipykernel_27036\3615335941.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disease_compound_new.drop('Ingredient_Smile',axis=1,inplace=True)


In [34]:
disease_herb['Herb_name'] = disease_herb['Herb_name'].apply(lambda x:x.split(';')[1])
disease_herb_new = disease_herb.query("Herb_name in @herb_all")

In [36]:
disease_herb_new.to_csv(r"D:\Herb数据库\version2\Positive_Disease_Herb_pair_in_HERB.csv")
disease_compound_new.to_csv(r"D:\Herb数据库\version2\Positive_Disease_Compound_pair_in_HERB.csv")